In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [3]:
soil_types = ['Red Soil', 'Black Clayey Soil', 'Brown Soil']

# Defining ranges for each property (example ranges, these can be adjusted with actual data)
soil_properties = {
    'Water Holding Capacity (mm/m)': {
        'Red Soil': (200, 300),
        'Black Clayey Soil': (350, 450),
        'Brown Soil': (250, 350)
    },
    'Field Capacity (%)': {
        'Red Soil': (20, 25),
        'Black Clayey Soil': (30, 40),
        'Brown Soil': (25, 35)
    },
    'Permanent Wilting Point (%)': {
        'Red Soil': (10, 15),
        'Black Clayey Soil': (15, 20),
        'Brown Soil': (12, 17)
    },
    'Infiltration Rate (mm/hour)': {
        'Red Soil': (1.5, 2.5),
        'Black Clayey Soil': (0.5, 1.0),
        'Brown Soil': (1.0, 2.0)
    },
    'Soil Texture Class (Sandy, Silty, Clay %)': {
        'Red Soil': (70, 10, 20),  # e.g., 70% sandy, 10% silty, 20% clay
        'Black Clayey Soil': (40, 20, 40),
        'Brown Soil': (50, 25, 25)
    },
    'Bulk Density (g/cm³)': {
        'Red Soil': (1.2, 1.4),
        'Black Clayey Soil': (1.3, 1.5),
        'Brown Soil': (1.2, 1.4)
    }
}


In [4]:

def generate_synthetic_data(soil_types, soil_properties, num_samples):
    data = []

    for _ in range(num_samples):
        soil_type = np.random.choice(soil_types)

        # Generate data for each property based on the soil type
        water_holding_capacity = np.random.uniform(*soil_properties['Water Holding Capacity (mm/m)'][soil_type])
        field_capacity = np.random.uniform(*soil_properties['Field Capacity (%)'][soil_type])
        wilting_point = np.random.uniform(*soil_properties['Permanent Wilting Point (%)'][soil_type])
        infiltration_rate = np.random.uniform(*soil_properties['Infiltration Rate (mm/hour)'][soil_type])

        # Soil texture (sandy, silty, clay %)
        sandy, silty, clay = soil_properties['Soil Texture Class (Sandy, Silty, Clay %)'][soil_type]

        # Generate bulk density
        bulk_density = np.random.uniform(*soil_properties['Bulk Density (g/cm³)'][soil_type])

        # Append the data as a row
        data.append([
            soil_type,
            water_holding_capacity,
            field_capacity,
            wilting_point,
            infiltration_rate,
            sandy, silty, clay,
            bulk_density
        ])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=[
        'Soil Type',
        'Water Holding Capacity (mm/m)',
        'Field Capacity (%)',
        'Permanent Wilting Point (%)',
        'Infiltration Rate (mm/hour)',
        'Sandy (%)', 'Silty (%)', 'Clay (%)',
        'Bulk Density (g/cm³)'
    ])

    return df

# Generate synthetic dataset
num_samples = 1000  # Define how many samples you want to generate
synthetic_data = generate_synthetic_data(soil_types, soil_properties, num_samples)
synthetic_data.head()


,Soil Type,Water Holding Capacity (mm/m),Field Capacity (%),Permanent Wilting Point (%),Infiltration Rate (mm/hour),Sandy (%),Silty (%),Clay (%),Bulk Density (g/cm³)
0,Black Clayey Soil,439.244233,30.729889,18.546972,0.816463,40,20,40,1.373647
1,Black Clayey Soil,378.790632,34.984335,16.513733,0.882883,40,20,40,1.326452
2,Brown Soil,288.470601,32.316135,13.189386,1.156767,50,25,25,1.264088
3,Red Soil,270.632063,20.314999,13.565113,2.228248,70,10,20,1.318909
4,Black Clayey Soil,417.464748,34.023248,16.514994,0.783501,40,20,40,1.377525


In [5]:
# GAN model setup
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dense(8, activation='tanh'))  # 8 values output for our soil data

    return model

def build_discriminator(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(512, input_shape=input_shape))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model


In [6]:
latent_dim = 100  # Size of the random input vector to the generator

# Build the generator and discriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator((8,))  # 8 features as input
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build and compile the GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile the discriminator separately
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# Compile the GAN by freezing the discriminator (no training for discriminator during GAN training)
gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
            loss='binary_crossentropy')


In [10]:
# Optimized training loop with smaller epochs and learning rate adjustment
def train_gan(epochs, batch_size, latent_dim, X_train, generator, discriminator, gan):
    batch_count = X_train.shape[0] // batch_size
    half_batch = batch_size // 2

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            real_samples = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_samples = generator.predict(noise)

            # Train discriminator on real and fake samples
            d_loss_real = discriminator.train_on_batch(real_samples, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator (we want to trick the discriminator into classifying fake samples as real)
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Label for generator is 1 (real)

        # Learning rate scheduler (reducing the learning rate over time)
        if epoch % 100 == 0:
            # Update the learning rate of the GAN's optimizer
            current_lr = gan.optimizer.learning_rate
            new_lr = current_lr * 0.95  # Reduce by 5% every 100 epochs
            gan.optimizer.learning_rate.assign(new_lr)  # Update the learning rate
            print(f"Epoch {epoch + 1}/{epochs} - Updated learning rate: {new_lr}")

        # Print training progress every few epochs
        if epoch % 500 == 0:
            print(f'{epoch+1}/{epochs} | D Loss: {d_loss[0]} | G Loss: {g_loss}')


In [11]:
# Train the GAN with optimized settings
epochs = 11   # Reduced epochs
batch_size = 32  # Smaller batch size for faster training
train_gan(epochs, batch_size, latent_dim, normalized_data, generator, discriminator, gan)


NameError: name 'normalized_data' is not defined

In [12]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming you have a DataFrame with the soil data (e.g., from a CSV file)
# Replace this with the actual data loading step
data = pd.read_csv('path_to_your_soil_data.csv')

# Normalize the numerical features (e.g., soil properties) using MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data.iloc[:, 1:])  # Exclude non-numeric columns (if any)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_soil_data.csv'

In [13]:
import pandas as pd

# Create sample data (you can replace this with your actual data)
data = {
    'SoilType': ['Red Soil', 'Black Clayey Soil', 'Brown Soil'],
    'WaterHoldingCapacity': [120, 140, 110],
    'FieldCapacity': [30, 35, 25],
    'PermanentWiltingPoint': [12, 15, 10],
    'InfiltrationRate': [5.2, 4.8, 5.5],
    'SoilTextureClass': ['Clay', 'Clay', 'Loam'],
    'BulkDensity': [1.3, 1.4, 1.35]
}

# Create a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = '/content/soil_data.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at: {csv_file_path}")


CSV file saved at: /content/soil_data.csv


In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming you have a DataFrame with the soil data (e.g., from a CSV file)
# Replace this with the actual data loading step
data = pd.read_csv('/content/soil_data.csv')

# Normalize the numerical features (e.g., soil properties) using MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data.iloc[:, 1:])  # Exclude non-numeric columns (if any)


ValueError: could not convert string to float: 'Clay'

In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Sample data (replace this with your actual data if you have a CSV file)
data = {
    'SoilType': ['Red Soil', 'Black Clayey Soil', 'Brown Soil'],
    'WaterHoldingCapacity': [120, 140, 110],
    'FieldCapacity': [30, 35, 25],
    'PermanentWiltingPoint': [12, 15, 10],
    'InfiltrationRate': [5.2, 4.8, 5.5],
    'SoilTextureClass': ['Clay', 'Clay', 'Loam'],
    'BulkDensity': [1.3, 1.4, 1.35]
}

# Create DataFrame
df = pd.DataFrame(data)

# Select only numeric columns for normalization
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Normalize the numeric columns
scaler = MinMaxScaler()
normalized_data = df.copy()  # Make a copy of the original dataframe

# Apply MinMaxScaler to the numeric columns only
normalized_data[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Check the normalized data
print(normalized_data)


            SoilType  WaterHoldingCapacity  FieldCapacity  \
0           Red Soil              0.333333            0.5   
1  Black Clayey Soil              1.000000            1.0   
2         Brown Soil              0.000000            0.0   

   PermanentWiltingPoint  InfiltrationRate SoilTextureClass  BulkDensity  
0                    0.4          0.571429             Clay          0.0  
1                    1.0          0.000000             Clay          1.0  
2                    0.0          1.000000             Loam          0.5  


In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MinMaxScaler

# Load and normalize the data (already done)
# Let's assume 'normalized_data' is available as the dataset

# Define the latent dimension (size of the noise vector)
latent_dim = 100

# Build the generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(7, activation='linear'))  # 7 features (adjust as necessary)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return model

# Build the discriminator
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output probability (real/fake)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator during GAN training

    # GAN input: random noise
    gan_input = Input(shape=(latent_dim,))
    x = generator(gan_input)

    # The discriminator evaluates the generated sample
    gan_output = discriminator(x)

    # Create the GAN model
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

    return gan

# Optimized training loop with smaller epochs and learning rate adjustment
def train_gan(epochs, batch_size, latent_dim, X_train, generator, discriminator, gan):
    batch_count = X_train.shape[0] // batch_size
    half_batch = batch_size // 2

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            real_samples = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_samples = generator.predict(noise)

            # Train discriminator on real and fake samples
            d_loss_real = discriminator.train_on_batch(real_samples, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator (we want to trick the discriminator into classifying fake samples as real)
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Label for generator is 1 (real)

        # Learning rate scheduler (reducing the learning rate over time)
        if epoch % 100 == 0:
            current_lr = gan.optimizer.learning_rate
            new_lr = current_lr * 0.95  # Reduce by 5% every 100 epochs
            gan.optimizer.learning_rate.assign(new_lr)  # Update the learning rate
            print(f"Epoch {epoch + 1}/{epochs} - Updated learning rate: {new_lr}")

        # Print training progress every few epochs
        if epoch % 500 == 0:
            print(f'{epoch+1}/{epochs} | D Loss: {d_loss[0]} | G Loss: {g_loss}')

# Instantiate and compile the models
generator = build_generator(latent_dim)
discriminator = build_discriminator(normalized_data.shape[1])  # Input shape should match the data
gan = build_gan(generator, discriminator)

# Train the GAN with optimized settings
epochs = 11   # You can experiment with different values (e.g., 1000 or 2000) based on your needs
batch_size = 32  # Smaller batch size for faster training

train_gan(epochs, batch_size, latent_dim, normalized_data, generator, discriminator, gan)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/11 - Updated learning rate: 0.00018999999156221747


UnboundLocalError: cannot access local variable 'd_loss' where it is not associated with a value

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MinMaxScaler

# Load and normalize the data (already done)
# Let's assume 'normalized_data' is available as the dataset

# Define the latent dimension (size of the noise vector)
latent_dim = 100

# Build the generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(7, activation='linear'))  # 7 features (adjust as necessary)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return model

# Build the discriminator
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output probability (real/fake)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator during GAN training

    # GAN input: random noise
    gan_input = Input(shape=(latent_dim,))
    x = generator(gan_input)

    # The discriminator evaluates the generated sample
    gan_output = discriminator(x)

    # Create the GAN model
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

    return gan

# Optimized training loop with smaller epochs and learning rate adjustment
def train_gan(epochs, batch_size, latent_dim, X_train, generator, discriminator, gan):
    batch_count = X_train.shape[0] // batch_size
    half_batch = batch_size // 2

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            real_samples = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_samples = generator.predict(noise)

            # Train discriminator on real and fake samples
            d_loss_real = discriminator.train_on_batch(real_samples, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((half_batch, 1)))

            # Correctly calculate d_loss as an average of the real and fake losses
            d_loss = 0.5 * np.add(d_loss_real[0], d_loss_fake[0])  # d_loss_real[0] and d_loss_fake[0] are loss values

            # Train Generator (we want to trick the discriminator into classifying fake samples as real)
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Label for generator is 1 (real)

        # Learning rate scheduler (reducing the learning rate over time)
        if epoch % 100 == 0:
            current_lr = gan.optimizer.learning_rate
            new_lr = current_lr * 0.95  # Reduce by 5% every 100 epochs
            gan.optimizer.learning_rate.assign(new_lr)  # Update the learning rate
            print(f"Epoch {epoch + 1}/{epochs} - Updated learning rate: {new_lr}")

        # Print training progress every few epochs
        if epoch % 500 == 0:
            print(f'{epoch+1}/{epochs} | D Loss: {d_loss} | G Loss: {g_loss}')

# Instantiate and compile the models
generator = build_generator(latent_dim)
discriminator = build_discriminator(normalized_data.shape[1])  # Input shape should match the data
gan = build_gan(generator, discriminator)

# Train the GAN with optimized settings
epochs = 5000    # You can experiment with different values (e.g., 1000 or 2000) based on your needs
batch_size = 32  # Smaller batch size for faster training

train_gan(epochs, batch_size, latent_dim, normalized_data, generator, discriminator, gan)


Epoch 1/5000 - Updated learning rate: 0.00018999999156221747


UnboundLocalError: cannot access local variable 'd_loss' where it is not associated with a value

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MinMaxScaler

# Load and normalize the data (already done)
# Let's assume 'normalized_data' is available as the dataset

# Define the latent dimension (size of the noise vector)
latent_dim = 100

# Build the generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(7, activation='linear'))  # 7 features (adjust as necessary)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return model

# Build the discriminator
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output probability (real/fake)

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator during GAN training

    # GAN input: random noise
    gan_input = Input(shape=(latent_dim,))
    x = generator(gan_input)

    # The discriminator evaluates the generated sample
    gan_output = discriminator(x)

    # Create the GAN model
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

    return gan

# Optimized training loop with smaller epochs and learning rate adjustment
def train_gan(epochs, batch_size, latent_dim, X_train, generator, discriminator, gan):
    batch_count = X_train.shape[0] // batch_size
    half_batch = batch_size // 2

    # Ensure the loss variables are initialized
    d_loss = 0
    g_loss = 0

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            real_samples = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_samples = generator.predict(noise)

            # Train discriminator on real and fake samples
            d_loss_real = discriminator.train_on_batch(real_samples, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((half_batch, 1)))

            # Correctly calculate d_loss as an average of the real and fake losses
            d_loss = 0.5 * np.add(d_loss_real[0], d_loss_fake[0])  # d_loss_real[0] and d_loss_fake[0] are loss values

            # Train Generator (we want to trick the discriminator into classifying fake samples as real)
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))  # Label for generator is 1 (real)

        # Learning rate scheduler (reducing the learning rate over time)
        if epoch % 100 == 0:
            current_lr = gan.optimizer.learning_rate
            new_lr = current_lr * 0.95  # Reduce by 5% every 100 epochs
            gan.optimizer.learning_rate.assign(new_lr)  # Update the learning rate
            print(f"Epoch {epoch + 1}/{epochs} - Updated learning rate: {new_lr}")

        # Print training progress every few epochs (only after d_loss and g_loss are computed)
        if epoch % 500 == 0:
            print(f'{epoch+1}/{epochs} | D Loss: {d_loss} | G Loss: {g_loss}')

# Instantiate and compile the models
generator = build_generator(latent_dim)
discriminator = build_discriminator(normalized_data.shape[1])  # Input shape should match the data
gan = build_gan(generator, discriminator)

# Train the GAN with optimized settings
epochs = 11    # Set the number of epochs to a smaller value for testing
batch_size = 32  # Smaller batch size for faster training

train_gan(epochs, batch_size, latent_dim, normalized_data, generator, discriminator, gan)


Epoch 1/11 - Updated learning rate: 0.00018999999156221747
1/11 | D Loss: 0 | G Loss: 0


In [19]:
# Generate random noise (latent space vectors)
num_samples = 1000  # The number of synthetic data samples you want to generate
noise = np.random.normal(0, 1, (num_samples, latent_dim))

# Generate synthetic samples using the trained generator
synthetic_samples = generator.predict(noise)

# Print the synthetic data generated
print(synthetic_samples)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
[[-0.11528634 -0.14168109  0.08321266 ...  0.15021764  0.0142203
  -0.17729372]
 [ 0.09401864 -0.18855149 -0.12699687 ...  0.18584234  0.05951953
   0.04435223]
 [-0.1264144  -0.17500928 -0.1420508  ...  0.13162531 -0.01237944
  -0.1169129 ]
 ...
 [-0.08167355 -0.2589823   0.09850308 ...  0.0763727   0.10813472
  -0.17228699]
 [-0.0898655  -0.48946387  0.13655443 ... -0.18606772  0.1520841
  -0.14812322]
 [-0.11362289 -0.20108548  0.10070263 ...  0.04314056 -0.03721219
  -0.04978454]]


In [22]:
print(normalized_data.shape)  # Check the shape of your normalized training data
print(synthetic_samples.shape)  # Check the shape of your generated samples


(3, 7)
(1000, 7)


In [26]:
import pandas as pd
from google.colab import files

# Assuming 'synthetic_samples' is the generated data from the generator
# Convert the synthetic data into a DataFrame
synthetic_data_df = pd.DataFrame(synthetic_samples, columns=['Soil type','Water Holding Capacity', 'Field Capacity',
                                                             'Permanent Wilting Point', 'Infiltration Rate',
                                                             'Soil Texture Class', 'Bulk Density'])

# Save the DataFrame as a CSV file
synthetic_data_df.to_csv('synthetic_soil_data.csv', index=False)

# Download the CSV file
files.download('synthetic_soil_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>